In [162]:
import numpy as np

In [163]:
def relu(Z):
  return np.maximum(0, Z)


def relu_backward(dA, Z):
  return np.where(Z > 0, dA, 0)


relu_backward(
    np.array([[0.1, 0.2], [-0.5, 2.1]]),
    np.array([[-5, 2], [1, -7]])
)

array([[ 0. ,  0.2],
       [-0.5,  0. ]])

In [164]:
def sigmoid(Z):
  return 1 / (1 + np.exp(-Z))


def sigmoid_backward(dA, Z):
  s = sigmoid(Z)
  return dA * s * (1 - s)


sigmoid(np.array([-5, -2, 2, 5]))

sigmoid_backward(
    np.array([-5, -2, 2, 5]),
    np.array([-1, 1, 0.5, -0.2]),
)

array([0.00669285, 0.11920292, 0.88079708, 0.99330715])

array([-0.98305967, -0.39322387,  0.47000742,  1.23758286])

In [165]:
def bce_loss(A2, y):
  A2 = np.clip(A2, 1e-9, (1 - 1e-9))
  loss = -np.mean(y*np.log(A2) + (1-y)*np.log(1-A2))
  return loss


def bce_loss_backward(A2, y):
  A2 = np.clip(A2, 1e-9, (1 - 1e-9))
  n = y.shape[0]
  return (-(y/A2) + (1-y)/(1-A2)) / n


bce_loss(
    A2=np.array([0.2, 0.55, 0.6, 0.1, 0.8]),
    y=np.array([0, 1, 1, 1, 0])
)

bce_loss(
    A2=np.array([0.2, 0.55, 0.6, 0.1, 0.8]),
    y=np.array([0, 1, 1, 0, 0])
)

np.float64(1.0487658362527934)

np.float64(0.6093209207855496)

In [166]:
A2 = np.array([0.2, 0.55, 0.6, 0.1, 0.8])
y = np.array([0, 1, 1, 1, 0])

In [167]:
bce_loss(A2, y)

lr = 0.01
dA2 = bce_loss_backward(A2, y)
A2 -= lr * dA2

np.float64(1.0487658362527934)

In [168]:
class LinearLayer:
  def __init__(self, n_inputs, n_neurons):
    self.W = np.random.randn(n_inputs, n_neurons) * 0.01
    self.b = np.zeros(n_neurons)

    self.X = None
    self.dW = None
    self.db = None

  def forward(self, X):
    self.X = X
    return X @ self.W + self.b

  def backward(self, dZ):
    self.dW = self.X.T @ dZ
    self.db = np.sum(dZ, axis=0)
    return dZ @ self.W.T

  def update(self, lr):
    self.W -= lr * self.dW
    self.b -= lr * self.db

In [169]:
class MLP:
  def __init__(self, n_inputs, n_hidden, n_outputs):
    self.layer1 = LinearLayer(n_inputs, n_hidden)
    self.layer2 = LinearLayer(n_hidden, n_outputs)

    self.Z1 = None
    self.Z2 = None
    self.A1 = None
    self.A2 = None

  def forward(self, X):
    self.Z1 = self.layer1.forward(X)
    self.A1 = relu(self.Z1)
    self.Z2 = self.layer2.forward(self.A1)
    self.A2 = sigmoid(self.Z2)
    return self.A2

  def backward(self, Y):
    dA2 = bce_loss_backward(self.A2, Y)
    dZ2 = sigmoid_backward(dA2, self.Z2)
    dA1 = self.layer2.backward(dZ2)
    dZ1 = relu_backward(dA1, self.Z1)
    self.layer1.backward(dZ1)

  def update(self, lr):
    self.layer1.update(lr)
    self.layer2.update(lr)

  def predict(self, X, threshold=0.5):
    probs = self.forward(X)
    return (probs >= threshold).astype(int)

In [170]:
def train(model, X, Y, lr=0.1, epochs=1000, print_every=100):
  loss_history = []

  initial_loss = bce_loss(model.predict(X), Y)
  print(f"Epoch {0:>4} | Loss: {initial_loss:.4f}")

  for epoch in range(1, epochs + 1):
    A2 = model.forward(X)

    loss = bce_loss(A2, Y)
    loss_history.append(loss)

    model.backward(Y)
    model.update(lr)

    if epoch % print_every == 0 or epoch == 1:
      print(f"Epoch {epoch:>4} | Loss: {loss:.4f}")

  return loss_history

In [171]:
# ============================================================
#  SCENARIO: Loan Approval
#
#  Same data as before but now we use a proper neural network.
#  Inputs: [income_lakhs, credit_score/100, existing_loans]
#  Label : 1 = approve, 0 = reject
# ============================================================

In [172]:
X = np.array([
    [3.0,  4.0,  2],
    [2.0,  3.5,  3],
    [4.0,  4.5,  1],
    [1.5,  3.0,  2],
    [5.0,  3.8,  3],
    [8.0,  7.5,  0],
    [10.0, 8.0,  1],
    [7.0,  7.0,  0],
    [12.0, 9.0,  0],
    [6.0,  7.2,  1],
])

Y = np.array([0, 0, 0, 0, 0, 1, 1, 1, 1, 1]).reshape(-1, 1)

model = MLP(3, 5, 1)

loss_history = train(model, X, Y)

Epoch    0 | Loss: 10.3616
Epoch    1 | Loss: 0.6932
Epoch  100 | Loss: 0.1853
Epoch  200 | Loss: 0.0698
Epoch  300 | Loss: 0.0473
Epoch  400 | Loss: 0.0345
Epoch  500 | Loss: 0.0259
Epoch  600 | Loss: 0.0200
Epoch  700 | Loss: 0.0159
Epoch  800 | Loss: 0.0129
Epoch  900 | Loss: 0.0108
Epoch 1000 | Loss: 0.0091


In [173]:
predictions = model.predict(X)
correct = (predictions == Y).sum()
accuracy = correct / len(Y) * 100
print(f"\nFinal Accuracy: {correct}/{len(Y)} = {accuracy:.1f}%")


Final Accuracy: 10/10 = 100.0%


In [174]:
print("\nDetailed Predictions:")
print(f"  {'Inputs':<30} {'Target':>8} {'Predicted':>10} {'Prob':>8}")
print("  " + "-" * 58)

probs = model.forward(X)
labels = ["reject", "reject", "reject", "reject", "reject",
          "approve", "approve", "approve", "approve", "approve"]

for i in range(len(X)):
  inp = str(X[i].tolist())
  target = int(Y[i][0])
  pred = int(predictions[i][0])
  prob = float(probs[i][0])
  status = "✅" if pred == target else "❌"
  print(f"  {inp:<30} {target:>8} {pred:>10} {prob:>8.3f}  {status}")


Detailed Predictions:
  Inputs                           Target  Predicted     Prob
  ----------------------------------------------------------
  [3.0, 4.0, 2.0]                       0          0    0.000  ✅
  [2.0, 3.5, 3.0]                       0          0    0.000  ✅
  [4.0, 4.5, 1.0]                       0          0    0.054  ✅
  [1.5, 3.0, 2.0]                       0          0    0.000  ✅
  [5.0, 3.8, 3.0]                       0          0    0.000  ✅
  [8.0, 7.5, 0.0]                       1          1    1.000  ✅
  [10.0, 8.0, 1.0]                      1          1    1.000  ✅
  [7.0, 7.0, 0.0]                       1          1    1.000  ✅
  [12.0, 9.0, 0.0]                      1          1    1.000  ✅
  [6.0, 7.2, 1.0]                       1          1    0.965  ✅


In [175]:
# ---- Test on unseen data ----
print("\nUnseen Applicants:")
X_new = np.array([
    [9.0,  8.5,  0],   # clearly good
    [2.5,  3.2,  4],   # clearly bad
    [6.5,  7.0,  1],   # borderline good
])
Y_new = np.array([1, 0, 1]).reshape(-1, 1)

preds_new = model.predict(X_new)
probs_new = model.forward(X_new)

for i in range(len(X_new)):
  inp = str(X_new[i].tolist())
  pred = int(preds_new[i][0])
  prob = float(probs_new[i][0])
  expected = int(Y_new[i][0])
  status = "✅" if pred == expected else "❌"
  print(f"  {inp:<30} predicted: {pred}  prob: {prob:.3f}  {status}")


Unseen Applicants:
  [9.0, 8.5, 0.0]                predicted: 1  prob: 1.000  ✅
  [2.5, 3.2, 4.0]                predicted: 0  prob: 0.000  ✅
  [6.5, 7.0, 1.0]                predicted: 1  prob: 0.969  ✅
